In [4]:
# author: Sylvain Laporte
# program: deepSDF_single_shape.py
# date: 2020-12-08
# object: Implementation of single-shape DeepSDF

# The PyTorch code structure used in this project is adapted from the tutorial "PyTorch: Zero to GANs" by Aakash N.S. for freeCodeCamp.org
# This tutorial is available at https://jovian.ai/aakashns/01-pytorch-basics

# Deep SDF - Single shape

In [ ]:
# # FOR COLAB. Installing packages
# !setup.py install

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import trimesh
import pyrender

## Preprocessing data

We need to sample 500 000 spatial points from a mesh and compute their corresponding SDF values. We sample more aggressively near the surface of the mesh.

### Preprocess the data

In [47]:
# Run from command line tool

### Using already preprocessed data

In [46]:
import json

# Load preprocessed dataset
dataset_file = "dataset/cube-samples.json"

points = None
sdfs = None

with open(dataset_file, "r") as f:
    data = json.load(f)
    points = data['points']
    sdfs = data['sdfs']

# Convert to a workable list of tuples
dataset = list(zip(points, sdfs))

### Split dataset in batches

#### Split in trainning dataset and validation dataset

In [49]:
validation_size = 10000
train_size = len(dataset) - validation_size

train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])
len(train_dataset), len(validation_dataset)

(240000, 10000)

#### Create batches

In [50]:
batch_size = 256

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(validation_dataset, batch_size*2, num_workers=4, pin_memory=True)

## Preparing for running on GPU

In [5]:
# Check if a CUDA GPU is available
torch.cuda.is_available()

False

In [ ]:
def get_default_device():
    """Helper function to select the device on which to run the model"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [ ]:
# Set and check the default device
device = get_default_device()
device

In [ ]:
def to_device(data, device):
    """Helper function to move the data to the chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader():
    """Dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)

### Load dataset to the GPU

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

## Training the neural network $f_\theta$

### Loss function

In [ ]:
def clamp(x, delta):
    return torch.min(delta, torch.max(-delta, x))

def loss(f_theta, s):
    delta = 0.1     # parameter that controls the distance from the surface
    return torch.abs(clamp(f_theta, delta) - clamp(s, delta))

### Model

#### Define the model

In [ ]:
def accuracy(outputs, sdf_values):
    """For human validation only"""
    pass

In [ ]:
class DeepSDF_single(nn.Module):
    """Multi-layer fully-connected feed-forward neural network with 8 layers, each applied with dropout"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        # Hidden layer 1
        self.linear1 = nn.linear(in_size, hidden_size)
        # Hidden layer 2
        self.linear2 = nn.linear(hidden_size, hidden_size)
        # Hidden layer 3
        self.linear3 = nn.linear(hidden_size, hidden_size)
        # Hidden layer 4
        self.linear4 = nn.linear(hidden_size, hidden_size)
        # Hidden layer 5
        self.linear5 = nn.linear(hidden_size, hidden_size)
        # Hidden layer 6
        self.linear6 = nn.linear(hidden_size, hidden_size)
        # Hidden layer 7
        self.linear7 = nn.linear(hidden_size, hidden_size)
        # Output layer
        self.linear8 = nn.linear(hidden_size, out_size)

    def forward(self, xb):
        out = self.linear1(xb)
        out = F.relu(out)
        out = self.linear2(xb)
        out = F.relu(out)
        out = self.linear3(xb)
        out = F.relu(out)
        out = self.linear4(xb)
        out = F.relu(out)
        out = self.linear5(xb)
        out = F.relu(out)
        out = self.linear6(xb)
        out = F.relu(out)
        out = self.linear7(xb)
        out = F.relu(out)
        out = self.linear8(xb)
        return out
    
    def training_step(self, batch):
        point_samples, sdf_values = batch
        out = self(point_samples)       # generate predictions
        loss = loss(out, sdf_values)    # compute loss
        return loss

    def validation_step(self, batch):
        point_samples, sdf_values = batch
        out = self(point_samples)           # generate predictions
        loss = loss(out, sdf_values)        # compute loss
        acc = accuracy(out, sdf_values)     # compute accuracy
        return {'val_loss': loss, 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # combine accuracies
        return {'val_loss': epoch_loss.items(), 'val_acc': epoch_acc.items()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

#### Create the model

In [2]:
input_size = 1
hidden_size = 512
out_size = 1

model = DeepSdf_single(input_size, hidden_size=hidden_size, out_size=out_size)

In [ ]:
# Check the parameters for each layers
for t in model.parameters():
    print(t.shape)

#### Load the model on the GPU

In [ ]:
to_device(model, device)

### Train the model

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters, lr)
    for epoch in range(epochs):
        # Training
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
# Checking the accuracy of the untrained model
history = [evaluate(model, val_loader)]
history

In [ ]:
epochs = 5
learning_rate = 0.5

# Train the model
history += fit(epochs, learning_rate, model, train_loader, val_loader)

### Plot the losses and accuracies

In [ ]:
# Plot the losses
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
losses = [x['val_loss'] for x in history]
plt.plot(losses, '-x')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss vs. No. of epochs')

accuracies = [x['val_acc'] for x in history]
plt.plot(accuracies, '-x')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy vs. No. of epochs')

## Render implicit surface

### Get the mesh

### Render the mesh